# oriC Finder & Bacterial Genome Analyzer

This notebook allows you to:
- Fetch bacterial genomes by NCBI accession
- Upload local FASTA files
- Paste genome sequences directly
- Calculate GC and AT skews (cumulative and sliding window)
- Scan for DnaA-box motifs using PWM
- Identify replication genes and annotate
- Visualize interactive plots with Plotly
- Export results to CSV and PNG

### Usage:
- Provide genome input (accession, upload, or paste)
- Adjust parameters (window size, PWM threshold)
- Run analysis and explore results

In [ ]:
# --- Cell 1: Imports and setup ---
import io
import os
import csv
import re
import requests
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# NCBI email required for Entrez (replace with your email)
Entrez.email = "your.email@example.com"  # <-- Replace this with your email for NCBI queries

# Global variables
genome_seq = None
genome_record = None
current_accession = None
motif_hits = []
fig_plot = None
genes = []

In [ ]:
# --- Cell 2: Functions for genome input ---

def fetch_genome_by_accession(accession):
    """Fetch genome from NCBI given accession number."""
    try:
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
        record = SeqIO.read(handle, "fasta")
        handle.close()
        return record
    except Exception as e:
        print(f"Error fetching accession {accession}: {e}")
        return None

def parse_fasta_file(file_contents):
    """Parse genome FASTA file contents (string) to SeqRecord."""
    try:
        handle = io.StringIO(file_contents)
        record = SeqIO.read(handle, "fasta")
        handle.close()
        return record
    except Exception as e:
        print(f"Error parsing FASTA: {e}")
        return None

def parse_pasted_sequence(seq_string):
    """Parse raw pasted sequence (may contain whitespace/newlines) to SeqRecord."""
    cleaned = re.sub(r"[^ATGCatgc]", "", seq_string)  # Remove non-ATGC characters
    seq_obj = Seq(cleaned.upper())
    from Bio.SeqRecord import SeqRecord
    record = SeqRecord(seq_obj, id="pasted_genome", description="User pasted genome sequence")
    return record

In [ ]:
# --- Cell 3: GC and AT skew calculation functions ---
def sliding_window_skew(seq, window=1000, step=100):
    """
    Calculate sliding window GC and AT skews.
    Returns list of positions and skews.
    """
    gc_skews = []
    at_skews = []
    positions = []
    seq_len = len(seq)
    for start in range(0, seq_len - window + 1, step):
        window_seq = seq[start:start+window]
        g = window_seq.count('G')
        c = window_seq.count('C')
        a = window_seq.count('A')
        t = window_seq.count('T')
        gc_skew = (g - c) / (g + c) if (g + c) > 0 else 0
        at_skew = (a - t) / (a + t) if (a + t) > 0 else 0
        gc_skews.append(gc_skew)
        at_skews.append(at_skew)
        positions.append(start + window // 2)
    return positions, gc_skews, at_skews

def cumulative_skew(seq):
    """
    Calculate cumulative GC and AT skew over the entire genome.
    """
    gc_skew_cum = []
    at_skew_cum = []
    g_count = 0
    c_count = 0
    a_count = 0
    t_count = 0
    for i, nt in enumerate(seq):
        if nt == 'G': g_count += 1
        elif nt == 'C': c_count += 1
        elif nt == 'A': a_count += 1
        elif nt == 'T': t_count += 1
        gc_skew_cum.append(g_count - c_count)
        at_skew_cum.append(a_count - t_count)
    return gc_skew_cum, at_skew_cum

In [ ]:
# --- Cell 4: PWM motif scanning for DnaA-box ---
def pwm_score(seq, pwm):
    """Calculate PWM score for a given sequence."""
    score = 0
    for i, nt in enumerate(seq):
        if nt in pwm[i]:
            score += pwm[i][nt]
        else:
            score += 0  # Penalize unknown nucleotides as 0
    return score

def scan_pwm(seq, pwm, threshold=5.0):
    """Scan sequence with PWM, return positions and scores above threshold."""
    hits = []
    pwm_len = len(pwm)
    seq = seq.upper()
    for i in range(len(seq) - pwm_len + 1):
        window_seq = seq[i:i+pwm_len]
        score = pwm_score(window_seq, pwm)
        if score >= threshold:
            hits.append((i, window_seq, score))
    return hits

# Define the DnaA-box PWM (based on E. coli consensus TTATCCACA, simplified example)
dnaA_pwm = [
    {'T': 1.0, 'A': 0.0, 'C': 0.0, 'G': 0.0},
    {'T': 0.5, 'A': 0.5, 'C': 0.0, 'G': 0.0},
    {'T': 0.0, 'A': 1.0, 'C': 0.0, 'G': 0.0},
    {'T': 0.0, 'A': 0.0, 'C': 1.0, 'G': 0.0},
    {'T': 0.0, 'A': 0.0, 'C': 1.0, 'G': 0.0},
    {'T': 0.5, 'A': 0.0, 'C': 0.0, 'G': 0.5},
    {'T': 0.0, 'A': 1.0, 'C': 0.0, 'G': 0.0},
    {'T': 0.0, 'A': 0.5, 'C': 0.0, 'G': 0.5},
    {'T': 0.0, 'A': 0.0, 'C': 1.0, 'G': 0.0}
]

In [ ]:
# --- Cell 5: Fetch replication genes from NCBI GenBank (annotations) ---
def fetch_replication_genes(accession):
    """
    Fetch annotated genes related to replication (e.g. dnaA, dnaN, dnaB) from NCBI GenBank.
    """
    try:
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
        replication_genes = []
        rep_gene_names = {'dnaA', 'dnaB', 'dnaC', 'dnaN', 'gyrB', 'recF', 'ssb'}
        for feature in record.features:
            if feature.type == 'gene' and 'gene' in feature.qualifiers:
                gene_name = feature.qualifiers['gene'][0].lower()
                if gene_name in rep_gene_names:
                    replication_genes.append({
                        'gene': gene_name,
                        'start': int(feature.location.start),
                        'end': int(feature.location.end),
                        'strand': feature.location.strand
                    })
        return replication_genes
    except Exception as e:
        print(f"Error fetching genes: {e}")
        return []

In [ ]:
# --- Cell 6: Widgets for user input and controls ---

# Accession input and fetch button
accession_input = widgets.Text(
    value='',
    placeholder='Enter NCBI accession (e.g. NC_000962)',
    description='Accession:',
    disabled=False
)
fetch_button = widgets.Button(description="Fetch Genome")

# Upload FASTA file
upload_button = widgets.FileUpload(
    accept='.fasta,.fa,.txt',
    multiple=False,
    description='Upload FASTA'
)

# Paste raw sequence
paste_area = widgets.Textarea(
    value='',
    placeholder='Paste raw genome sequence here (ATGC only)',
    description='Paste Seq:',
    layout=widgets.Layout(width='80%', height='100px')
)

# Parameters sliders
window_slider = widgets.IntSlider(value=5000, min=1000, max=20000, step=1000, description='Window Size:')
pwm_threshold_slider = widgets.FloatSlider(value=5.0, min=1.0, max=10.0, step=0.1, description='PWM Threshold:')

# Run analysis button
run_button = widgets.Button(description='Run Analysis', button_style='success')

# Export buttons
export_csv_button = widgets.Button(description='Export Motif Hits CSV')
export_plot_button = widgets.Button(description='Export Plot PNG')

# Output display
output_area = widgets.Output()

In [ ]:
# --- Cell 7: Main analysis and plotting logic ---
def run_analysis(b):
    global genome_seq, genome_record, motif_hits, genes, fig_plot, current_accession
    with output_area:
        clear_output()
        if genome_seq is None:
            print("No genome sequence loaded. Please fetch, upload, or paste a genome.")
            return
        
        window = window_slider.value
        threshold = pwm_threshold_slider.value
        print(f"Running analysis with window size={window}, PWM threshold={threshold}...")
        
        # Calculate skews
        positions, gc_skews, at_skews = sliding_window_skew(genome_seq, window=window, step=window//10)
        gc_skew_cum, at_skew_cum = cumulative_skew(genome_seq)
        
        # PWM scanning
        motif_hits = scan_pwm(str(genome_seq), dnaA_pwm, threshold=threshold)
        print(f"Found {len(motif_hits)} DnaA-box motif hits above threshold.")
        
        # Fetch replication genes annotations if accession known
        genes = []
        if current_accession is not None:
            genes = fetch_replication_genes(current_accession)
            print(f"Found {len(genes)} replication-related genes in annotation.")
        else:
            print("No accession known; skipping gene annotation.")
        
        # Plot
        fig = go.Figure()
        # Plot cumulative GC skew
        fig.add_trace(go.Scatter(x=list(range(len(gc_skew_cum))), y=gc_skew_cum, mode='lines', name='Cumulative GC Skew'))
        # Plot cumulative AT skew
        fig.add_trace(go.Scatter(x=list(range(len(at_skew_cum))), y=at_skew_cum, mode='lines', name='Cumulative AT Skew'))
        
        # Mark motif hits on plot
        motif_x = [pos for pos, seq_, score in motif_hits]
        motif_y = [gc_skew_cum[pos] if pos < len(gc_skew_cum) else 0 for pos in motif_x]
        fig.add_trace(go.Scatter(x=motif_x, y=motif_y, mode='markers', name='DnaA-box Motifs', marker=dict(color='red', size=6)))
        
        # Mark replication genes
        for gene in genes:
            gene_pos = (gene['start'] + gene['end']) // 2
            if gene_pos < len(gc_skew_cum):
                fig.add_trace(go.Scatter(x=[gene_pos], y=[gc_skew_cum[gene_pos]], mode='markers+text',
                                         name=f"Gene: {gene['gene']}", marker=dict(symbol='star', size=10, color='purple'),
                                         text=[gene['gene']], textposition='bottom right'))
        
        fig.update_layout(title="Genome Skew Analysis & DnaA-box Motifs",
                          xaxis_title="Genome Position (bp)",
                          yaxis_title="Skew / Score",
                          height=600)
        fig.show()
        
        fig_plot = fig

In [ ]:
# --- Cell 8: Callbacks for inputs and exports ---
def on_fetch_clicked(b):
    global genome_seq, genome_record, current_accession
    with output_area:
        clear_output()
        acc = accession_input.value.strip()
        if not acc:
            print("Please enter a valid NCBI accession number.")
            return
        print(f"Fetching genome for accession: {acc} ...")
        record = fetch_genome_by_accession(acc)
        if record:
            genome_seq = record.seq.upper()
            genome_record = record
            current_accession = acc
            print(f"Genome loaded: {record.id} ({len(record.seq)} bp)")
        else:
            print("Failed to fetch genome.")

def on_upload_change(change):
    global genome_seq, genome_record, current_accession
    with output_area:
        clear_output()
        if upload_button.value:
            for fname, fileinfo in upload_button.value.items():
                content = fileinfo['content'].decode('utf-8')
                record = parse_fasta_file(content)
                if record:
                    genome_seq = record.seq.upper()
                    genome_record = record
                    current_accession = None
                    print(f"Genome loaded from file: {fname} ({len(record.seq)} bp)")
                else:
                    print("Error parsing uploaded file.")
            upload_button.value.clear()
            upload_button._counter = 0

def on_paste_submit(change):
    global genome_seq, genome_record, current_accession
    with output_area:
        clear_output()
        seq_text = paste_area.value.strip()
        if not seq_text:
            print("Paste area is empty.")
            return
        record = parse_pasted_sequence(seq_text)
        genome_seq = record.seq.upper()
        genome_record = record
        current_accession = None
        print(f"Genome loaded from pasted sequence ({len(record.seq)} bp)")

def export_motif_hits_csv(b):
    if not motif_hits:
        with output_area:
            print("No motif hits to export.")
        return
    filename = "motif_hits.csv"
    with open(filename, "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Position", "Motif Sequence", "Score"])
        for pos, seq_, score in motif_hits:
            writer.writerow([pos, seq_, score])
    with output_area:
        print(f"Motif hits exported to {filename}")

def export_plot_png(b):
    global fig_plot
    if fig_plot is None:
        with output_area:
            print("No plot to export.")
        return
    filename = "genome_skew_plot.png"
    fig_plot.write_image(filename)
    with output_area:
        print(f"Plot exported to {filename}")

# Link buttons
fetch_button.on_click(on_fetch_clicked)
upload_button.observe(on_upload_change, names='value')
paste_area.observe(on_paste_submit, names='value')
run_button.on_click(run_analysis)
export_csv_button.on_click(export_motif_hits_csv)
export_plot_button.on_click(export_plot_png)

In [ ]:
# --- Cell 9: Display interface ---
display(widgets.VBox([
    widgets.Label("### Genome Input"),
    accession_input, fetch_button,
    widgets.Label("Or upload a FASTA file:"),
    upload_button,
    widgets.Label("Or paste sequence directly:"),
    paste_area,
    widgets.Label("### Parameters"),
    window_slider,
    pwm_threshold_slider,
    run_button,
    widgets.HBox([export_csv_button, export_plot_button]),
    output_area
]))